# Decorators

## Concepts

- Python's answer to C macros, Java annotations, C# attributes, and JavaScript decorators
- Most similar to JavaScript decorators (interpreted runtime behavior)
- Wraps one piece of code with another
- Supports a terse @ syntax for composing higher-order functions
- Defines function that takes another function and returns a new function
- Used to extend behavior of a function without explicitly modifying it
- Used for cross-cutting concerns
- Aspect-Oriented Programming 
- Decorators vs. the Decorator Pattern

[Python Decorators - a short animation](https://www.youtube.com/watch?v=iHNZrXZj-p8)

## Topics

- Functional Programming
- Simple Decorator Function
- Decorator Syntactic Sugar
- Stacking Decorators
- Invocation Counter
- Parameterized Decorator Function
- Decorator Functions with ```*args``` and ```**kwargs```
- Function Call Rate Limiting
- Authorization and Access Control
- Logging and Instrumentation
- Memoization Performance Enhancement
- Decorator Classes
- The ``` @staticmethod``` Decorator
- The ``` @classmethod``` Decorator
- The ``` @property``` Decorator
- The ``` @contextmanager``` Decorator
- The ``` @functools.wraps``` Decorator
- The ``` @functools.lru_cache``` Decorator
- Function Timeout Example (Decorator Library)
- Singleton (Decorator Library)
- Synchronization (Decorator Library)

## Functional Programming
- Functions
- Function Objects
- Lambdas
- Higher Order Functions

In [3]:
# define function
def my_func_1(x):
    print("my_func_1 called")
    return x**2

# directly call function by name
result = my_func_1 (4)
print(result) # output: 16

# indirectly call function via function object
f = my_func_1
result = f(5)
print(result) # output: 25

# pass function object as a function argument
def call_function(f, x):
    return f(x)
result = call_function(f, 6)
print(result) # output: 36

# return function object as a function result
def return_function(x):
    return lambda : x * x    # lambda expression but could have used a named function
print('calling returned lambda function')
result = return_function(7)()
print(result) # output: 49

my_func_1 called
16
my_func_1 called
25
my_func_1 called
36
calling returned lambda function
49


## Simple Decorator Function

- Decorator function
- Nested return function
- Decorated target function
- Extends functionality via code injection

In [4]:
# simple decorator function
def my_decorator_2(old_func):
    print("my_decorator_2 called")    
    def new_func(n):                     # nested function
        print("new_func called")
        return old_func(n)               # invoke original target function
    return new_func                      # return nested function

# target function to be decorated
def my_func_2(x):
    print("my_func_2 called")
    return x**2

result = my_func_2(4)                    # call function before decoration
print(result)
print("my_func_2.__name__ -> ", my_func_2.__name__)    # -> my_func_2

print('---')

my_func_2 = my_decorator_2(my_func_2)    # programmatically decorate function
result = my_func_2(4)                    # call function after decoration
print(result)
print("my_func_2.__name__ -> ", my_func_2.__name__)    # -> new_func

# NOTE: The change to __name__ issue can be fixed using @wraps from functools (See later)

my_func_2 called
16
my_func_2.__name__ ->  my_func_2
---
my_decorator_2 called
new_func called
my_func_2 called
16
my_func_2.__name__ ->  new_func


## Decorator Syntactic Sugar

- Uses the declarative ```@``` decorator syntax
- Automatically sets up higher order function logic as shown in previous example
- Extends functionality via dynamic code injection

In [5]:
# decorator function
def my_decorator_3(old_func):
    print("my_decorator_3 called")
    def new_func(n):
        print("new_func called")
        return old_func(n)
    return new_func

# decorator syntactic sugar
@my_decorator_3                     # declaratively decorate function
def my_func_3(x):
    print("my_func_3 called")
    return x**2
    
result = my_func_3(4)
print(result)

my_decorator_3 called
new_func called
my_func_3 called
16


## Stacking Decorators

- You can stack or chain decorators
- Order determined by sequence of decorators

In [5]:
def decorator_A(old_func):
    def new_func():
        print("decorator_A called")
        return old_func()
    return new_func

def decorator_B(old_func):
    def new_func():
        print("decorator_B called")
        return old_func()
    return new_func

@decorator_A
@decorator_B
def funcAB():
    print("funcAB called")

# The above two decorators are equivalent to:
# funcAB = decorator_A(decorator_B(funcAB))

funcAB()

decorator_A called
decorator_B called
funcAB called


# Invocation Counter

- Uses a closure to store a free variable as persistent data in inner function lexical scope
- Function call counter is initialized to zero
- Function call counter is incremented on each call to the decorated function

In [7]:
# invocation counter decorator
def invocation_counter(old_func):
    counter = 0                     # closure
    def new_func(n):
        nonlocal counter
        counter += 1
        print("counter: ", counter)
        return old_func(n)
    return new_func

# decorator syntactic sugar
@invocation_counter
def square(x):
    return x**2
    
result = print(square(4))
result = print(square(5))
result = print(square(6))

counter:  1
16
counter:  2
25
counter:  3
36


## Parameterized Decorator Function

- Parameter is used to provide decorator with initial value
- Any number of parameters of any type could be provided
- The parameter value is stored in a closure for future use
- The parameter value is displayed on subsequent invocations for demonstration purposes
- The decorator is applied to the target function with an argument provided

In [10]:
# parameterized decorator function
def my_decorator_4(text):
    print("my_decorator_4 called with parameter:", text)
    def wrap_old_func(old_func):
        print("wrap_old_func called")
        def new_func(n):
            print("new_func called")
            print(text)
            return old_func(n)
        return new_func
    return wrap_old_func

# decorator syntactic sugar
@my_decorator_4("Hello World!")    # decorator with parameter
def my_func_4(x):
    print("my_func_4 called")
    return x**2
    
result = my_func_4(4)
print(result)

my_decorator_4 called with parameter: Hello World!
wrap_old_func called
new_func called
Hello World!
my_func_4 called
16


## Decorator Functions with *args and **kwargs

- ```*args``` allows target function to take any number of non-keyworded arguments
- ```**kwargs``` allows target function to take any number of keyworded arguments

In [6]:
# Review: Variadic Functions

def afunction(a, b, c, *args, **kwargs):
    print("parameters: ", a, b, c, args, kwargs )
    
afunction(1, 2, 3)                               # required -> 3 positional arguments
afunction(1, 2, 3, 4, 5, 6)                      # optional -> non-keyword arguments
afunction(1, 2, 3, x=7, y=8, z=9)                # optional -> keyword arguments
afunction(1, 2, 3, 4, 5, 6, x=7, y=8, z=9)       # optional -> keyword and non-keyword arguments
afunction(*(1, 2, 3), *(4, 5, 6))                            # unpacking list
afunction(*(1, 2, 3), **{'x':7 , 'y':8 , 'z':9})             # unpacking dictionary
afunction(*(1, 2, 3), *(4, 5, 6), **{'x':7 , 'y':8 , 'z':9}) # unpacking both

parameters:  1 2 3 () {}
parameters:  1 2 3 (4, 5, 6) {}
parameters:  1 2 3 () {'x': 7, 'y': 8, 'z': 9}
parameters:  1 2 3 (4, 5, 6) {'x': 7, 'y': 8, 'z': 9}
parameters:  1 2 3 (4, 5, 6) {}
parameters:  1 2 3 () {'x': 7, 'y': 8, 'z': 9}
parameters:  1 2 3 (4, 5, 6) {'x': 7, 'y': 8, 'z': 9}


In [2]:
# parameterized decorator function taking *args and **kwargs
def my_decorator_5(original_func):
    print("my_decorator_5 called")
    def wrapper_func(*args, **kwargs):
        print("wrapper_func called")
        return original_func(*args, **kwargs)
    return wrapper_func

@my_decorator_5                  # declaratively decorate function
def my_func_5(name, age):
    print("my_func_5 called")
    return name + "'s age is " + str(age)
result = my_func_5('Jane', 42)    # output: Jane's age is 42
print(result)

print('---')

@my_decorator_5                               # declaratively decorate function
def my_other_func_5(a, b, c, x= 0, y = 0, z = 0):
    print("my_other_func_5 called")
    return a + b + c + x + y + z
result = my_other_func_5(1, 2, 3, x=4, y=5)   # output: 15
print(result)

my_decorator_5 called
wrapper_func called
my_func_5 called
Jane's age is 42
---
my_decorator_5 called
wrapper_func called
my_other_func_5 called
15


## Function Call Rate Limiting

- A decorator named limit_rate is defined with a parameter named min_time
- A closure is created containing a variable named prev_time initialized as time.clock() - min_time
- Every time the decorated function is called, curr_time is established
- If time transpired is less than min_time then prev_time is updated and None is returned
- Otherwise prev_time is updated and the result of the target function is returned
- If the decorated function is called repeatedly in rapid succession it ignores the call
- If the decorated function is  called repeatedly with a sufficient delay it makes the call

In [16]:
import time

# limit rate decorator
def limit_rate(min_time):
    def wrap_old_func(old_func):
        prev_time = time.clock() - min_time  # closure
        def new_func(n):
            nonlocal prev_time
            curr_time = time.clock()
            print(curr_time - prev_time)
            if curr_time - prev_time < min_time:
                prev_time = curr_time
                return None
            else:
                prev_time = curr_time
                return old_func(n)
        return new_func
    return wrap_old_func

# decorator syntactic sugar
@limit_rate(1.0)             # consecutive calls must be at least 1 second apart
def square(x):
    return x**2
    
result = print(square(4))    # output: 16
result = print(square(5))    # output: None
result = print(square(6))    # output: None
time.sleep(2.0)
result = print(square(7))    # output: 49
result = print(square(8))    # output: None
result = print(square(9))    # output: None

1.000079874663712
16
0.0004078516348201771
None
0.00017135123388506348
None
2.012211899004512
49
0.0004239158129966114
None
0.0001664427349981068
None


# Authorization and Access Control

- This is a toy example that focuses more on decorators than proper security coding
- Initially no user is logged in and the global authorized variable is set to False
- The login function is hard-coded for demonstration purposes
- The logout function is hard-coded for demonstration purposes
- The require_auth decorator forces its target function to require authentication
- If authorized is True then the DoSomethingDangerous executes its code normally
- If authorized is False then the DoSomethingDangerous does not execute its code

In [18]:
authorized = False

def login(username, password):    
    global authorized
    if username == 'Carole' and password == 'Pa55w.rd':
        authorized = True
        print('login succeeded')
    else:
        print('login failed')

def logout():
    global authorized
    print('logout called')
    authorized = False
    
def require_auth(func):
    def decorated_func(*args, **kwargs):
        if not authorized:
            print('ERROR: Not authorized')
            return None
        return func(*args, **kwargs)
    return decorated_func

@require_auth
def DoSomethingDangerous():
    print('DoSomethingDangerous called with authorized:', authorized)

DoSomethingDangerous()          # output: ERROR: Not authorized
print('---')
login('Carole', 'Pa55w.rd')
DoSomethingDangerous()          #  output: DoSomethingDangerous called with authorized: True
print('---')
logout()
DoSomethingDangerous()          #  output: ERROR: Not authorized
print('---')
login('David', 'wrong-guess')
DoSomethingDangerous()          #  output: ERROR: Not authorized
print('---')
login('Carole', 'Pa55w.rd')
DoSomethingDangerous()          # output: DoSomethingDangerous called with authorized: True

ERROR: Not authorized
---
login succeeded
DoSomethingDangerous called with authorized: True
---
logout called
ERROR: Not authorized
---
login failed
ERROR: Not authorized
---
login succeeded
DoSomethingDangerous called with authorized: True


# Logging and Instrumentation

- This is a toy logging example that focuses more on decorators than proper logging code
- For a proper discussion on logging see: [Logging HOWTO](https://docs.python.org/3/howto/logging.html)

In [19]:
import datetime
import time

def logging(func):
    def func_with_logging(*args, **kwargs):
        print(func.__name__ + " args: " + str(args) + " kwargs" + str(kwargs) + " called at: " + str(datetime.datetime.now()))
        return func(*args, **kwargs)
    return func_with_logging

@logging
def addition_func(x, y=0):
   """Do some math."""
   return x + y

print(addition_func(4))
time.sleep(1)
print(addition_func(5))
time.sleep(1)
print(addition_func(6, 33))
time.sleep(1)
print(addition_func(6, y = 42))

addition_func args: (4,) kwargs{} called at: 2018-04-26 19:46:14.864120
4
addition_func args: (5,) kwargs{} called at: 2018-04-26 19:46:15.872258
5
addition_func args: (6, 33) kwargs{} called at: 2018-04-26 19:46:16.873464
39
addition_func args: (6,) kwargs{'y': 42} called at: 2018-04-26 19:46:17.881763
48


## Memoization and Performance Enhancement

- This is a toy caching example that focuses more on decorators than proper caching 
- See the example elsewhere in this lesson on the ``` @functools.lru_cache``` decorator
- Memoization is an optimization technique that leverages cached results
- Memoization can speed up some algorithms by avoiding costly redundant calculations
- This example implements a custom timeit decorator for demonstration purposes…
-     .... but see: [timeit](https://docs.python.org/3/library/timeit.html)

In [3]:
def memoize(func):
    cache = func.cache = {}
    def memoized_func(*args, **kwargs):
        key = str(args) + str(kwargs)
        if key not in cache:
            cache[key] = func(*args, **kwargs)
        return cache[key]
    return memoized_func

import time

def timeit(method):
    def timed(*args, **kw):
        t_start = time.time()
        result = method(*args, **kw)
        t_end = time.time()
        #print (f"{method.__name__} -> {t_end-t_start} seconds") # requires Python 3.6
        print ("{} -> {} seconds".format(method.__name__, t_end-t_start))
        return result
    return timed

@memoize
def fibonacci_memoized(n):    # O(n) -> time complexity is linear (each result for 0 to n only once)
    if n == 0: return 0
    if n == 1: return 1
    else: return fibonacci_memoized(n-1) + fibonacci_memoized(n-2)

def fibonacci_plain(n):       # O(2^n) -> time complexity is exponential
    if n == 0:return 0
    if n == 1:return 1
    else: return fibonacci_plain(n-1) + fibonacci_plain(n-2)

@timeit
def call_fibonacci_plain(n):
    return fibonacci_plain(n)

print("Wait for result...")

result = call_fibonacci_plain(35)
print("    call_fibonacci_plain(35):", result)

# = call_fibonacci_plain(50)  # takes long time then blows the stack
#print("    call_fibonacci_plain(50):", result)

@timeit
def call_fibonacci_memoized(n):
    return fibonacci_memoized(n)

result = call_fibonacci_memoized(35)
print("    call_fibonacci_memoized(35):", result)

result = call_fibonacci_memoized(35)
print("    call_fibonacci_memoized(35):", result)

result = call_fibonacci_memoized(50)
print("    call_fibonacci_memoized(50):", result)

result = call_fibonacci_memoized(100)
print("    call_fibonacci_memoized(100):", result)

Wait for result...
call_fibonacci_plain -> 11.299617528915405 seconds
    call_fibonacci_plain(35): 9227465
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(35): 9227465
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(35): 9227465
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(50): 12586269025
call_fibonacci_memoized -> 0.0 seconds
    call_fibonacci_memoized(100): 354224848179261915075


## Popular Pre-built Decorators

There are many commonly used decorators, including the following:

- ``` @staticmethod```
- ``` @classmethod```
- ``` @proprty```
- ``` @contextmanager```
- ``` @functools.wraps```
- ``` @functools.lru_cache```

## The ``` @staticmethod``` Decorator

- The ``` @staticmethod``` decorator modifies a method function so that it does not use the self variable
- The method function will not have access to a specific instance of the class
- Declarative alternative to calling the ```staticmethod()``` function

In [21]:
class BankAccount:
    interest = 0.0
    def __init__(self, balance):
        self.balance = balance
    def transact(self, amount):
        self.balance += amount
    def getBalance(self):
        return self.balance
    @staticmethod
    def setInterest(rate):
        BankAccount.interest = rate
    @staticmethod
    def getInterest(rate):
        return BankAccount.interest
    def accrueInterest(self):
        self.balance *= (1 + BankAccount.interest)

ba = BankAccount(100.0)
print(ba.getBalance())
ba.accrueInterest()
print(ba.getBalance())
BankAccount.setInterest(0.01)
print(BankAccount.getInterest(0.01))
ba.accrueInterest()
print(ba.getBalance())
ba.transact(1000)
print(ba.getBalance())
ba.accrueInterest()
print(ba.getBalance())

100.0
100.0
0.01
101.0
1101.0
1112.01


## The ``` @classmethod``` Decorator

- The ``` @classmethod``` decorator modifies method to receives class object as first parameter
- Does not use self to refer to class instance
- Used to implement singleton pattern
- Often used for introspection of a class
- Often used internally within many Python frameworks
- Declarative alternative to calling the ```classmethod()``` function

In [22]:
import pprint

class MyClassyClass:
    staticval = 42
    @classmethod
    def printClassObject(cls):
        print(cls)
    @classmethod
    def printClassVars(cls):
        pprint.pprint(cls.__dict__) # pretty-print on the attributes

MyClassyClass.printClassObject()
print(MyClassyClass.staticval)
MyClassyClass.printClassVars()

<class '__main__.MyClassyClass'>
42
mappingproxy({'__dict__': <attribute '__dict__' of 'MyClassyClass' objects>,
              '__doc__': None,
              '__module__': '__main__',
              '__weakref__': <attribute '__weakref__' of 'MyClassyClass' objects>,
              'printClassObject': <classmethod object at 0x00000218FCB71550>,
              'printClassVars': <classmethod object at 0x00000218FCB711D0>,
              'staticval': 42})


## The ``` @property``` Decorator

- Based on the ```property()``` function that creates an access-controlled attribute
- Supports *set*, *get* and *delete* operations

In [23]:
class UltimateQuestion:
    def __init__(self):
        self._answer = 42

    @property
    def answer(self):
        """Get the answer to the ultimate question."""
        return self._answer

uq = UltimateQuestion()
print("The answer to the ultimate question: ", uq.answer)
#uq.answer = "To get to the other side" # AttributeError: can't set attribute
#del uq.answer                          # AttributeError: can't delete attribute

print('---')

class Person:
    def __init__(self, nickname):
        self._nickname = nickname

    @property
    def nickname(self):
        """Get nickname."""
        return self._nickname

    @nickname.setter
    def nickname(self, nickname):
        """Set nickname."""
        self._nickname = nickname

    @nickname.deleter
    def nickname(self):
        """Delete nickname."""
        del self._nickname

person = Person("Rocky")
print(person.nickname)    # can get attribute
person.nickname = "Spike" # can set attribute
print(person.nickname)
del person.nickname       # can delete attribute
#print(person.nickname)    # AttributeError: 'Person' object has no attribute '_nickname'

The answer to the ultimate question:  42
---
Rocky
Spike


## The ``` @contextmanager``` Decorator

- A context manager is used to properly manage resources even when exception is raised
- The ```with``` statement replaces a ```try...finally``` block to ensure clean-up code is executed
- Used when two related operations must execute as a pair before and after a block of code

In [4]:
from contextlib import contextmanager

# decorate a generator function, that calls yield
@contextmanager
def open_file(path, mode):
    file = open(path, mode)
    try:
        yield file
    finally:
        file.close()

files = []

# automatically closes file when loop terminates
try:
    with open_file('junk.txt', 'w') as f:
        for x in range(20):
            print(x)
            if x == 13:
                raise Exception('13 is just unlucky!')
            f.write(str(x) + '\n')
except Exception as ex:
    print(ex)

if f.closed:
    print('All Good: file was automatically closed :)')
else:
    print('Big Problem: file not closed :(')

0
1
2
3
4
5
6
7
8
9
10
11
12
13
13 is just unlucky!
All Good: file was automatically closed :)


## The functools module ``` @wraps``` Decorator

- The ``` @wraps``` decorator is a part of the functools module
- Use the wraps decorator to fix docstrings and names of decorated functions
- Fixes introspection problems for decorated functions that would give wrong information

In [26]:
def not_cool_decorator_function(func):
    """Docstring: not_cool_decorator_function"""
    def wrapper():
        """Docstring: wrapper"""
        print('not_cool_decorator_function called')
        func()
    return wrapper

@not_cool_decorator_function
def not_cool_target_function():
    """Docstring: not_cool_target_function"""
    print('not_cool_target_function called')

not_cool_target_function()
print ("not_cool_target_function.__name__ ->", not_cool_target_function.__name__)
print ("not_cool_target_function.__doc__ ->", not_cool_target_function.__doc__)

print()

from functools import wraps

def cool_decorator_function(func):
    """Docstring: cool_decorator_function"""
    @wraps(func)
    def wrapper():
        """Docstring: wrapper"""
        print('cool_decorator_function called')
        func()
    return wrapper

@cool_decorator_function
def cool_target_function():
    """Docstring: cool_target_function"""
    print('cool_target_function called')

cool_target_function()
print ("cool_decorator_function.__name__ ->", cool_decorator_function.__name__)
print ("cool_decorator_function.__doc__ ->", cool_decorator_function.__doc__)


not_cool_decorator_function called
not_cool_target_function called
not_cool_target_function.__name__ -> wrapper
not_cool_target_function.__doc__ -> Docstring: wrapper

cool_decorator_function called
cool_target_function called
cool_decorator_function.__name__ -> cool_decorator_function
cool_decorator_function.__doc__ -> Docstring: cool_decorator_function


## The functools module ``` @lru_cache``` Decorator

- Cache memoizing to improve performance
- Saves up to the maxsize most recent calls

In [29]:
import functools

@functools.lru_cache(maxsize=None)
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

print([fibonacci(n) for n in range(20)])
print(fibonacci.cache_info())

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181]
CacheInfo(hits=36, misses=20, maxsize=None, currsize=20)


## Decorator Classes

- We have seen that a function can be used to define a decorator...
- But more generally, a decorator can be any callable object that takes a function as a parameter
- Therefore, a class that implements the ```__call__``` method can also be used to define a decorator 

In [13]:
# Callable Class Decorator
class PrintEnterExit(object):
    def __init__(self, func):
        self.func = func

    def __call__(self):
        print("Entering:", self.func.__name__)
        self.func()
        print("Exited:", self.func.__name__)
        print("---")

@PrintEnterExit
def func1():
    print("In: func1()")

@PrintEnterExit
def func2():
    print("In: func2()")

func1()
func2()

Entering: func1
In: func1()
Exited: func1
---
Entering: func2
In: func2()
Exited: func2
---


## Descriptor Class Decorator

- This is a toy example that focuses more on decorators than typical property coding
- See the example elsewhere in this lesson on using the ``` @property``` decorator instead
- A descriptor is an object attribute with “binding behavior”
- Applies attributes (i.e. properties) to the decorated target class
- Can implement getter, setter, and deleter using the ```__get__```, ```__set__```, and ```__delete__``` methods

In [8]:
# descriptor class decorator
class myproperty(object):
    def __init__(self, func):
        self.func = func
        self.name = func.__name__
    def __get__(self, instance, owner):
        #print(f'Getting {self.name} property') # requires Python 3.6
        print('Getting {} property'.format(self.name))
        return instance.__dict__[self.name]
    def __set__(self, instance, value):
        #print('Setting {self.name} property to {value}') # requires Python 3.6
        print('Setting {} property to {}'.format(self.name, value))
        instance.__dict__[self.name] = value
        
# target class
class Person():
    @myproperty
    def firstname(self):
        pass
    @myproperty
    def age(self):
        pass
    
person = Person()

person.age = 42
print("    person.age ->", person.age)
person.age = 102
print("    person.age ->", person.age)

print('---')
person.firstname = 'Sally'
print("    person.firstname ->", person.firstname)
person.firstname = 'Jane'
print("    person.firstname ->", person.firstname)

Setting age property to 42
Getting age property
    person.age -> 42
Setting age property to 102
Getting age property
    person.age -> 102
---
Setting firstname property to Sally
Getting firstname property
    person.firstname -> Sally
Setting firstname property to Jane
Getting firstname property
    person.firstname -> Jane


## Python Decorator Library

See: [Python Decorator Library at wiki.python.org](https://wiki.python.org/moin/PythonDecoratorLibrary)

## Function Timeout (Decorator Library)

- Note: POSIX only (signal.SIGALRM and signal.alarm() not available on Windows)
- See: https://wiki.python.org/moin/PythonDecoratorLibrary#Function_Timeout
- If you use Windows then you can use repl.it to run on Linux
- See: https://repl.it/@peterteach/CrazyFabulousDeletions

```python
import signal
import functools

class TimeoutError(Exception): pass

def timeout(seconds, error_message = 'Function call timed out'):
    def decorated(func):
        def _handle_timeout(signum, frame):
            raise TimeoutError(error_message)

        def wrapper(*args, **kwargs):
            signal.signal(signal.SIGALRM, _handle_timeout) # POSIX only
            signal.alarm(seconds)                          # POSIX only
            try:
                result = func(*args, **kwargs)
            finally:
                signal.alarm(0)
            return result

        return functools.wraps(func)(wrapper)

    return decorated

import time

@timeout(1, 'Function slow -> aborted')
def slow_function(n):
    time.sleep(n)
    return 42

print(slow_function(0.01))
print(slow_function(0.5))
try:
  print(slow_function(2))
except TimeoutError as ex:
  print(ex)
print(slow_function(0.75))
```

============================================

```
Python 3.6.1 (default, Dec 2015, 13:05:11)
[GCC 4.8.2] on linux
   
42
42
Function slow -> aborted
42
```

## Singleton (Decorator Library)

See: [wiki.python.org Python Decorator Library Singleton](https://wiki.python.org/moin/PythonDecoratorLibrary#Singleton)  

In [9]:
# See: https://wiki.python.org/moin/PythonDecoratorLibrary#Singleton

import functools

def singleton(cls):
    ''' Use class as singleton. '''

    cls.__new_original__ = cls.__new__

    @functools.wraps(cls.__new__)
    def singleton_new(cls, *args, **kw):
        it =  cls.__dict__.get('__it__')
        if it is not None:
            return it

        cls.__it__ = it = cls.__new_original__(cls, *args, **kw)
        it.__init_original__(*args, **kw)
        return it

    cls.__new__ = singleton_new
    cls.__init_original__ = cls.__init__
    cls.__init__ = object.__init__

    return cls

#
# Sample use:
#

@singleton
class Foo:
    def __new__(cls):
        cls.x = 10
        return object.__new__(cls)

    def __init__(self):
        assert self.x == 10
        self.x = 15

assert Foo().x == 15
Foo().x = 20
assert Foo().x == 20

## Synchronization (Decorator Library)

See: [wiki.python.org Python Decorator Library Synchronization](https://wiki.python.org/moin/PythonDecoratorLibrary#Synchronization)

In [10]:
# See: https://wiki.python.org/moin/PythonDecoratorLibrary#Synchronization

def synchronized(lock):
    '''Synchronization decorator.'''

    def wrap(f):
        def new_function(*args, **kw):
            lock.acquire()
            try:
                return f(*args, **kw)
            finally:
                lock.release()
        return new_function
    return wrap

# Example usage:

from threading import Lock
my_lock = Lock()

@synchronized(my_lock)
def critical1(*args):
    # Interesting stuff goes here.
    pass

@synchronized(my_lock)
def critical2(*args):
    # Other interesting stuff goes here.
    pass

In [11]:
import sys
print (sys.version)

3.6.4 |Anaconda, Inc.| (default, Jan 16 2018, 10:22:32) [MSC v.1900 64 bit (AMD64)]
